In [8]:
import pandas as pd
import numpy as np
import json
import hunspell
from datasketch import MinHash, MinHashLSH
from nltk import ngrams
hobj = hunspell.HunSpell('../French.dic', '../French.aff')
hobj.add('cdd')
hobj.add('cdi')
hobj.add('rtt')
hobj.add('ass')
hobj.add('csp')
hobj.add('smic')

0

In [70]:
with open('./cdtn_entities_last.json', encoding="utf-8") as f:
    lines = f.readlines()
entities = [json.loads(line) for line in lines]
entities_df = pd.DataFrame(entities)

In [74]:
most_popular = entities_df[entities_df.value > 3]

In [79]:
acceptable_length = most_popular[most_popular.entity.apply(lambda x : len(x.split(' ')) < 10)]

In [91]:
acceptable_length.to_csv("crawled_suggestions.csv")

In [97]:
'mal aux yeux' in entities_df.entity.tolist()

False

In [94]:
acceptable_length[:50]

,entity,value
0,licenciement,1294
1,rupture conventionnelle,966
2,congés payés,935
3,contrat de travail,823
4,licenciement pour inaptitude,809
5,démission,730
6,retraite,690
7,licenciement économique,612
8,licenciement pour faute grave,589
9,solde de tout compte,577


In [92]:
def suggestion_filter(sugg):
    for token in sugg.split(' '):
        if not hobj.spell(token):
            return False
        else :
            return True

In [4]:
spellchecked_entities = entities_df[entities_df["entity"].apply(lambda x: suggestion_filter(x))]

In [6]:
most_frequent = spellchecked_entities[spellchecked_entities.value > 3]

In [44]:
lsh = MinHashLSH(threshold=0.8, num_perm=128)

minhashes = {}
for c, i in enumerate(most_frequent['entity']):
    minhash = MinHash(num_perm=128)
    for d in ngrams(i, 3):
        minhash.update("".join(d).encode('utf-8'))
    lsh.insert(c, minhash)
    minhashes[c] = minhash

In [53]:
sets = []

unambiguous = []

def search_candidate(doc):
    minhash = MinHash(num_perm=128)
    for d in ngrams(doc, 3):
        minhash.update("".join(d).encode('utf-8'))

    search = lsh.query(minhash)
    
    if (len(search) > 1):
        print("Disambiguation match : " + doc)
        print([most_frequent.iloc[i].entity for i in search])
        print()
        sets.append(search)
    else:
        unambiguous.append(doc)

In [54]:
# print ambiguous
most_frequent['entity'].apply(lambda x : search_candidate(x))

Disambiguation match : licenciement
['licenciement', 'le licenciement']

Disambiguation match : rupture conventionnelle
['rupture conventionnelle', 'la rupture conventionnelle', 'rupture conventionnelle et formation', 'inaptitude ou rupture conventionnelle', 'une rupture conventionnelle', 'rupture conventionnelle urgent', 'rupture conventionnelle cadre']

Disambiguation match : congés payés
['congés payés', 'congés payés cdd']

Disambiguation match : contrat de travail
['contrat de travail cdi', 'contrat de travail à 39 h', 'contrat de travail', 'contrat de travail illégal', 'mon contrat de travail', 'contrat de travail nuit', "signature d'un contrat de travail", 'contrat de travail etranger']

Disambiguation match : licenciement pour inaptitude
['licenciement pour inaptitude', 'démission ou licenciement pour inaptitude']

Disambiguation match : retraite
['retraite', 'retraite forcée']

Disambiguation match : licenciement économique
['licenciement économique et csp', 'licenciement écon

Disambiguation match : durée préavis
['duré de préavis', 'durée préavis']

Disambiguation match : départ retraite
['prime départ retraite', 'départ retraite']

Disambiguation match : rupture conventionnelle ou licenciement économique
['rupture conventionnelle ou licenciement économique', 'rupture conventionnelle ou licenciement économique dans mon cas', 'licenciement économique ou rupture conventionnelle']

Disambiguation match : congé individuel de formation
['congé individuel de formation', 'congés payés et congé individuel de formation']

Disambiguation match : apprentissage
['apprentis', 'apprentissage', 'apprentie']

Disambiguation match : bulletin de salaire
['bulletin de salaire', 'le bulletin de salaire']

Disambiguation match : congés pour mariage
['congés pour mariage', 'congés pour mon mariage']

Disambiguation match : prime de panier
['prime de panier de jour', 'prime de panier']

Disambiguation match : modulation du temps de travail
['annulation temps de travail', 'modulat

Disambiguation match : rupture contrat cdd
['rupture contrat cdd', 'rupture contrat cui']

Disambiguation match : congés payés cdd
['congés payés', 'congés payés cdd']

Disambiguation match : démission pendant arrêt maladie
['démissionner pendant mon arrêt maladie', 'démission pendant arrêt maladie']

Disambiguation match : besoin de renseignement
['besoin de renseignement', 'besoin de renseignements svp']

Disambiguation match : avenant a mon contrat
['avenant a mon contrat', 'avenant à un contrat']

Disambiguation match : cdd transformé en cdi
['cdi transformé en cdd', 'cdd transformé en cdi']

Disambiguation match : prime de transport
['titre de transport', 'prime de transport']

Disambiguation match : dispositions de la présente
['dispositions de la présente section', 'dispositions de la présente']

Disambiguation match : contrat de travail à durée indéterminée
['contrat de travail à durée déterminée', 'contrat de travail à durée indéterminée']

Disambiguation match : contrôle de i

Disambiguation match : licenciement économique pendant un arrêt maladie
['licenciement économique en arrêt maladie', 'licenciement économique pendant un arrêt maladie']

Disambiguation match : demande de conseils
['demande de conseils', 'demande de cp']

Disambiguation match : rupture période d'essai et délai de prévenance
["période d'essai délai de prévenance", "rupture période d'essai et délai de prévenance"]

Disambiguation match : travail sans contrat de travail
['sans contrat de travail', 'travail sans contrat de travail']

Disambiguation match : indemnisation congés payés
['indemnisation congés payés', 'congés payés indemnisation']

Disambiguation match : reprise d'activité
["baisse d'activité", "reprise d'activité"]

Disambiguation match : congés payés non payés
['congés payés non pris', 'congés payés non payés']

Disambiguation match : salaire apprenti
['salaire apprenti', 'salaire apprentissage']

Disambiguation match : démission rupture conventionnelle
['démission rupture con

Disambiguation match : prime de départ
['prime de départ', 'prime départ']

Disambiguation match : pas de convention collective
['pas de convention collective', 'salaire convention collective', 'quelle convention collective', 'n de convention collective']

Disambiguation match : formation syndicale
['information syndicale', 'formation syndicale']

Disambiguation match : nombre de jours de travail
['poste de travail', 'nombre de jours de travail', 'durée de travail', 'lieu de travail', 'milieu de travail', 'nombre de jours travaillés', 'dispense de travail', 'pas de travail', 'fin de travail', 'prime de travail']

Disambiguation match : arrêt maladie et complément de salaire
['maladie et complément de salaire', 'arrêt maladie et complément de salaire']

Disambiguation match : conditions exercice du travail à temps partiel
['conditions exercice du temps partiel', 'conditions exercice du travail à temps partiel']

Disambiguation match : prise en charge formation
['prise en charge formatio

...

Disambiguation match : relevé carrière
['relevé carrière', 'relevé de carrière']

Disambiguation match : agent fonctionnaire ou contractuel
['fonctionnaire ou agent contractuel', 'agent fonctionnaire ou contractuel']

Disambiguation match : prime départ
['prime de départ', 'prime départ']

Disambiguation match : nombre de jours travaillés
['nombre de jours de travail', 'nombre de jours travaillés']

Disambiguation match : fonctionnaire ou agent contractuel
['fonctionnaire ou agent contractuel', 'agent fonctionnaire ou contractuel']

Disambiguation match : réduction du temps de travail rtt
['réduction de temps de travail', 'réduction du temps de travail rtt']

Disambiguation match : information syndicale
['information syndicale', 'formation syndicale']

Disambiguation match : service autonome de santé au travail
['service de santé au travail', 'service autonome de santé au travail']

Disambiguation match : missions inspection du travail
["l' inspection du travail", 'missions inspection 

Disambiguation match : signature d'un avenant au contrat de travail
["signature d'un avenant au contrat de travail", "modification de l'avenant au contrat de travail", 'avenant au contrat de travail']

Disambiguation match : indemnité chômage ou départ en retraite
['indemnité de départ en retraite', 'indemnité chômage ou départ en retraite']

Disambiguation match : indemnisation csp
['indemnisation', 'indemnisation csp']

Disambiguation match : calculs congés payés
['congés payés calcul', 'calculs congés payés']

Disambiguation match : calcul indemnités rupture conventionnelle
['calcul indemnités rupture conventionnelle', 'indemnités rupture conventionnelle', 'rupture conventionnelle et indemnité', "calcul de l'indemnité de rupture conventionnelle"]

Disambiguation match : période de congés payés
['période de congés payés', 'solde de congés payés']

Disambiguation match : reprise prime sur salaire
['reprise prime sur salaire', 'reprise sur salaire']

Disambiguation match : reprise sur 

Disambiguation match : problème avec mon ancien employeur
['problème avec mon employeur', 'problème avec mon ancien employeur']

Disambiguation match : versement de salaire
['versement de salaire', 'non versement de salaires']

Disambiguation match : versement retraite complémentaire
['versement retraite complémentaire', 'paiement retraite complémentaire']

Disambiguation match : modification contrat de travail cdi
['modification contrat de travail cdi', 'modification de mon contrat de travail']

Disambiguation match : modification de travail
['modification de travail', 'modification lieu de travail', 'modification des jours de travail']

Disambiguation match : modification de l'avenant au contrat de travail
["signature d'un avenant au contrat de travail", "modification de l'avenant au contrat de travail", 'avenant au contrat de travail']

Disambiguation match : solde de tout compte erroné
['solde de tout compte', 'solde de tout compte erroné']

Disambiguation match : erreur sur certif

Disambiguation match : délai de réponse rupture conventionnelle
['rupture conventionnelle délai de réponse', 'délai de réponse rupture conventionnelle']

Disambiguation match : délai paiement heures supplémentaires
['délai paiement heures supplémentaires', 'paiement des heures supplémentaires']

Disambiguation match : délai de carence cdd
['délai de carence', 'délai de carence cdd']

Disambiguation match : ouverture droits
['ouverture de droit', 'ouverture droits']

Disambiguation match : promesse d'embauche rompu
["promesse d'embauche rompu", "promesse d'embauche"]

Disambiguation match : baisse de salaire net
['baisse de salaire net', 'baisse de salaire']

Disambiguation match : baisse de la retraite
['baisse de la retraite', 'réponse de la caisse de retraite']

Disambiguation match : ai je droit au licenciement
['ai je droit au licenciement', 'droit au licenciement']

Disambiguation match : comment obtenir une rupture conventionnelle
['obtenir une rupture conventionnelle', 'comment 

0       None
1       None
2       None
3       None
4       None
        ... 
9065    None
9066    None
9067    None
9068    None
9069    None
Name: entity, Length: 8852, dtype: object

In [58]:
acc = []

for s in sets:
    se = set(s)
    match = None
    for ses in acc:
        if len(se.intersection(ses)) > 0:
            match = ses
            break
    if (match != None):
        acc.remove(match)
        acc.append(match&se)
    else :
        acc.append(se)

In [61]:
# randomly choose between ambiguous
disambiguated = []
for a in acc :
    disambiguated.append(most_frequent.iloc[list(a)[0]].entity)

In [69]:
pd.Series(unambiguous + disambiguated).to_csv("crawled_suggestions.csv")

/cdtn/ubuntu-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
